In [57]:
import pandas as pd
import os
from shapely.geometry import Point, Polygon

cwd =os.getcwd()

### Structure socioeconomic data by year

In [14]:
socioeco_df = pd.read_excel(cwd+'/socioeconomic_data/socioeco_data.xlsx', sheet_name = 3)

years = [2011, 2015, 2019]
socec_split = [socioeco_df[socioeco_df['YEAR'] == year] for year in years]

### Merge school data and socioeconomic data by postal code and kommune

In [16]:
df_list = []
for data, year in zip(socec_split, [2011, 2015, 2019]):
    # load data
    soc_data = data
    school_data = pd.read_csv(cwd+f'/school_data/school_clean_{year-1}-{year}.csv', index_col=0)

    # merge data
    merged = pd.merge(left = school_data, right = soc_data, left_on = ['beliggenhedskommunenr', 'postnr'], right_on = ['KOMKODE', 'POSTNR'])

    # format
    merged.columns = merged.columns.str.lower()
    merged = merged.T.drop_duplicates().T

    merged.to_csv(f'background_vars{year}.csv')

    df_list.append(merged)

In [17]:
all_years = pd.concat(df_list)
all_years.to_csv('background_all_years.csv')

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pyproj

ModuleNotFoundError: No module named 'geopandas'

In [109]:
df = pd.read_csv('background_vars2019.csv', index_col=0)
c = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]

gdf = gpd.GeoDataFrame(df, geometry = c)
gdf.crs = {'init': 'epsg:4326'}
gdf.to_crs(epsg = 32633)

RuntimeError: b'no arguments in initialization list'